In [ ]:
google_colab = True
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)
    base_path = '/content/drive/MyDrive/MasterThesis/'
else:
    base_path = '../.'

Mounted at /content/drive


In [ ]:
if google_colab:
    !pip install socceraction matplotsoccer
    !pip install pandas
    !pip install tables
    !pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.5 MB/s 
     |████████████████████████████████| 197 kB 17.4 MB/s 
  Created wheel for matplotsoccer: filename=matplotsoccer-0.0.8-py3-none-any.whl size=5983 sha256=57dd9cfbc293bbbe2a9d639fbfbd45a47061c025697e79ceb6ba10dbade72506
  Stored in directory: /root/.cache/pip/wheels/be/6f/64/335f5a5e7763b284f2f7d8893f20338de4d9f73794fe85ea7d
Successfully built matplotsoccer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 1.2 MB/s 


In [ ]:

import os
import json
import warnings
import shutil
import json
import pandas as pd
from matplotlib import pyplot as plt
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from enum import Enum

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
import socceraction.vaep.formula as vaepformula
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import nn
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss
from sklearn.metrics import roc_curve, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import LabelEncoder

import plotly.graph_objects as go
import plotly.figure_factory as ff

if google_colab:
    % load_ext autoreload
    % autoreload 2
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))


class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

1
Tesla P100-PCIE-16GB


In [ ]:
if google_colab:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = "cpu"
seed = 42


def seed_new_generator():
    return torch.Generator().manual_seed(seed)


os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True
    print(torch.cuda.current_device())

0


In [ ]:
if google_colab:
    os.system(f'git clone https://github.com/statsbomb/open-data {base_path}')

In [ ]:
if google_colab:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

    from psutil import virtual_memory

    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Fri Aug 12 12:27:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class StatsBombDataset(Enum):
    BARCELONA = 1
    WOMEN = 2

In [ ]:
dataset_version = StatsBombDataset.BARCELONA

if dataset_version == StatsBombDataset.BARCELONA:
    datafolder = os.path.join(base_path, "data-barcelona/")
elif dataset_version == StatsBombDataset.WOMEN:
    datafolder = os.path.join(base_path, "data-women/")

archive_path = "./"

spadl_h5 = os.path.join(datafolder, "spadl-statsbomb.h5")
print(os.path.exists(spadl_h5))
features_h5 = os.path.join(datafolder, "features.h5")
labels_h5 = os.path.join(datafolder, "labels.h5")
predictions_h5 = os.path.join(datafolder, "predictions.h5")
vaep_values_h5 = os.path.join(datafolder, "vaep_values.h5")

neural_features_h5 = os.path.join(datafolder, "neural_features.h5")
neural_predictions_h5 = os.path.join(datafolder, "neural_predictions.h5")
substitutions_h5 = os.path.join(datafolder, "substitutions.h5")

player_features_h5 = os.path.join(datafolder, "player_features.h5")
player_substitutions_h5 = os.path.join(datafolder, "player_substitutions.h5")
player_list_per_season_json = os.path.join(datafolder, "player_list_per_season.json")
player_list_name_per_season_json = os.path.join(datafolder, "player_list_name_per_season.json")
cumsum_current_player_stats_npy = os.path.join(datafolder, "cumsum_current_player_stats.npy")
current_player_stats_npy = os.path.join(datafolder, "cumulated_player_stats.npy")
cumsum_history_player_stats_npy = os.path.join(datafolder, "cumsum_history_player_stats.npy")
context_game_stats_npy = os.path.join(datafolder, "context_game_stats.npy")
history_player_stats_npy = os.path.join(datafolder, "history_player_stats.npy")
player_lineups_npy = os.path.join(datafolder, "player_lineups")
game_substitutions_npy = os.path.join(datafolder, "game_substitutions")
home_player_lineups_npy = os.path.join(datafolder, "home_player_lineups")
away_player_lineups_npy = os.path.join(datafolder, "away_player_lineups")
home_game_substitutions_npy = os.path.join(datafolder, "home_game_substitutions")
away_game_substitutions_npy = os.path.join(datafolder, "away_game_substitutions")

model_predictions_npy = os.path.join(datafolder, "model_predictions.npy")

SBL = StatsBombLoader(root=os.path.join(base_path, 'data'), getter='local')

assert os.path.exists(datafolder)
print(home_game_substitutions_npy)
# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)

mode = "player"

True
/content/drive/MyDrive/MasterThesis/data-barcelona/home_game_substitutions


In [ ]:
games = pd.read_hdf(spadl_h5, "games")

max_minutes = 0
player_list_per_season = {}
barcelona_id = 217

seasons = pd.read_hdf(spadl_h5, "competitions")
sorted_seasons = seasons.sort_values('season_name', ascending=True)
game_id_list = []
for season in seasons.itertuples():
    season_games = games[games.season_id == season.season_id]
    sorted_season_games = season_games.sort_values('game_day', ascending=True)
    for game in sorted_season_games.itertuples():
        game_id_list.append(int(game.game_id))

with open(player_list_per_season_json, 'r') as fp:
    player_list_per_season = json.load(fp)
    max_minutes = player_list_per_season['max_minutes']


player_id_encoder = LabelEncoder()

player_list = pd.DataFrame(columns=['player_id', 'player_name'])
for season in seasons.itertuples():
    if dataset_version == StatsBombDataset.BARCELONA:
        season_players = pd.read_json(player_list_per_season[str(season.season_id)])
    else:
        season_players = pd.DataFrame(columns=['player_id', 'player_name'])
        for team_id in player_list_per_season[str(season.season_id)].keys():
            team_players = pd.read_json(player_list_per_season[str(season.season_id)][team_id])
            season_players = pd.concat([season_players, team_players]).drop_duplicates().reset_index(drop=True)
    player_list = pd.concat([player_list, season_players]).drop_duplicates().reset_index(drop=True)

full_player_list = np.array(list(player_list['player_id'])).astype(int)
full_player_name_list = np.array(list(player_list['player_name']))
encoded_player_ids = player_id_encoder.fit_transform(full_player_list)
player_id_to_index = dict(zip(full_player_list, encoded_player_ids))
id_dict = dict(zip(encoded_player_ids, full_player_list))
name_dict = dict(zip(encoded_player_ids, full_player_name_list))

In [ ]:
starting_XI_type_id = 35
player_on_bench_index = 0
player_on_field_index = 1
player_in_index = 0
player_out_index = 1
nb_sub_batches = 3

In [ ]:
with open(current_player_stats_npy, 'rb') as fp:
    current_player_stats = np.load(fp)
with open(cumsum_current_player_stats_npy, 'rb') as fp:
    cumsum_current_player_stats = np.load(fp)
with open(context_game_stats_npy, 'rb') as fp:
    context_game_stats = np.load(fp)

In [ ]:
with open(history_player_stats_npy, 'rb') as fp:
    history_player_stats = np.load(fp)
with open(history_player_stats_npy, 'rb') as fp:
    cumsum_history_player_stats = np.load(fp)

In [ ]:
if dataset_version == StatsBombDataset.BARCELONA:
    game_substitutions = np.load(game_substitutions_npy + '.npz')
    game_line_ups = np.load(player_lineups_npy + '.npz')
else:
    game_substitutions = np.load(game_substitutions_npy + '.npz')
    game_line_ups = np.load(player_lineups_npy + '.npz')

In [ ]:
game_list = []
for season in seasons.itertuples():
    season_games = games[games.season_id == season.season_id]
    sorted_season_games = season_games.sort_values('game_day', ascending=True)
    for game in tqdm(sorted_season_games.itertuples()):
        game_list.append(int(game.game_id))
if dataset_version == StatsBombDataset.WOMEN :
    game_list = np.tile(game_list, 2)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
class MaskMode(Enum):
    MAX_MINUTE_SIZE = 1
    FIXED = 2
    LAST_MINUTE = 3


class MaskedPlayerDataset(Dataset):
    def __init__(self,
                 game_indices,
                 game_id_list,
                 player_substitutions_h5,
                 cumulated_player_stats,
                 expected_player_stats,
                 context_game_stats,
                 mask_mode,
                 mask_size=10):

        self.game_indices = game_indices
        self.game_id_list = np.array(game_id_list)
        self.mask_mode = mask_mode
        self.mask_size = mask_size
        self.num_games = len(self.game_indices)
        self.num_minutes = expected_player_stats.shape[1]
        self.num_players = expected_player_stats.shape[2]
        self.num_features = expected_player_stats.shape[3]
        self.context_num_features = context_game_stats.shape[2]

        self.player_expected_stats = torch.from_numpy(expected_player_stats).to(device)
        self.player_cumulated_stats = torch.from_numpy(cumulated_player_stats).to(device)
        self.context_stats = torch.from_numpy(context_game_stats).to(device)
        self.time_substitutions = torch.zeros(self.num_games, self.num_minutes, nb_sub_batches, 1).to(device)
        self.player_substitutions = torch.zeros(self.num_games, self.num_minutes, nb_sub_batches, 2).long().to(device)
        self.player_lineups_mask = torch.zeros(
            (self.num_games, self.num_minutes, nb_sub_batches, 2, self.num_players)).to(device)

        for i, game_id in enumerate(tqdm(self.game_id_list, "Extracting player substitutions")):
            if dataset_version == StatsBombDataset.BARCELONA:
                substitutions = game_substitutions[f'{game_id}']
                game_line_up = game_line_ups[f'{game_id}']
            else:
                if i < (len(self.game_id_list) / 2):
                    substitutions = game_substitutions[f'{game_id}/home']
                    game_line_up = game_line_ups[f'{game_id}/home']
                else:
                    substitutions = game_substitutions[f'{game_id}/away']
                    game_line_up = game_line_ups[f'{game_id}/away']
            time_sub = torch.unsqueeze(torch.from_numpy(substitutions[:, :, 0]), dim=-1)
            substitution_times, substitution_batches = np.where(substitutions[:, :, 0] > 0)
            player_in = substitutions[substitution_times, substitution_batches][:, player_in_index + 1]
            player_out = substitutions[substitution_times, substitution_batches][:, player_out_index + 1]
            for j, sub_time in enumerate(substitution_times):
                self.player_substitutions[i][sub_time][substitution_batches[j]][player_in_index] = int(player_in[j])
                self.player_substitutions[i][sub_time][substitution_batches[j]][player_out_index] = int(player_out[j])
            self.time_substitutions[i] = time_sub
            self.player_lineups_mask[i] = torch.from_numpy(game_line_up)

        self.masks = []
        if self.mask_mode == MaskMode.MAX_MINUTE_SIZE or self.mask_mode == MaskMode.LAST_MINUTE:
            for i in range(self.num_minutes - 2):
                self.masks.append(torch.LongTensor([0, i + 2, i + 2]))
        elif self.mask_mode == MaskMode.FIXED:
            for i in range(self.num_minutes - self.mask_size):
                self.masks.append(torch.LongTensor([i, i + self.mask_size, self.mask_size]))
        self.masks = torch.stack(self.masks, dim=0).to(device)
        self.item_list = torch.arange(self.num_games).repeat(self.masks.shape[0]).to(device)
        self.masks = torch.repeat_interleave(self.masks, self.num_games, dim=0)

    def __len__(self):
        return self.item_list.shape[0]

    def __getitem__(self, item):
        game_index = self.item_list[item]
        mask_item = self.masks[item]

        time_mask_length = mask_item[2].clone().detach().to('cpu')
        minute = mask_item[1].clone().detach().to('cpu') - 1
        time_mask = torch.zeros(self.num_minutes).type(torch.bool).to(device)
        time_mask[mask_item[0]:mask_item[1]] = True

        player_expected_stats = torch.zeros(self.num_minutes, self.num_players, self.num_features).type(
            torch.double).to(device)
        player_expected_stats[time_mask] = self.player_expected_stats[game_index][time_mask]

        player_cumulated_stats = torch.zeros(self.num_minutes, self.num_players, self.num_features).type(
            torch.double).to(device)
        player_cumulated_stats[time_mask] = self.player_cumulated_stats[game_index][time_mask]

        context_stats = torch.zeros(self.num_minutes, self.context_num_features).type(
            torch.double).to(device)
        context_stats[time_mask] = self.context_stats[game_index][time_mask]

        time_substitutions = torch.zeros(self.num_minutes, nb_sub_batches, 1).to(device)
        time_substitutions[time_mask, :] = self.time_substitutions[game_index][time_mask]

        player_lineup_mask = torch.zeros(self.num_minutes, nb_sub_batches, 2, self.num_players).type(torch.float).to(
            device)
        player_lineup_mask[time_mask, :] = self.player_lineups_mask[game_index][time_mask]

        player_substitutions = self.player_substitutions[game_index, minute]
        time_mask = torch.unsqueeze(time_mask, dim=-1)
        return item, game_index, minute, player_expected_stats.float(), player_cumulated_stats.float(), context_stats.float(), time_substitutions.float(), time_mask.int(), time_mask_length, player_substitutions.long(), player_lineup_mask.float()

    def get_game_id(self, item):
        return self.game_id_list[item]


class PlayerSubDataset(Dataset):
    def __init__(self, player_dataset):
        self.player_dataset = player_dataset
        self.items = []
        for i in tqdm(range(len(self.player_dataset)), "Building player sub dataset"):
            item, game_indices, minutes, _, _, _, time_sub, _, time_mask_length, _, _ = self.player_dataset[i]
            if torch.any(time_sub[time_mask_length - 1] == 1):
                self.items.append(i)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, item):
        return self.player_dataset[self.items[item]]

In [ ]:

game_id_copy = np.array(game_list.copy())
game_indices = list(range(len(game_id_copy)))
print(len(game_indices))
mask_mode = MaskMode.MAX_MINUTE_SIZE
#time_dataset = MaskedPlayerDataset(
#    game_indices,
#    game_id_copy,
#    game_substitutions,
#    current_player_stats,
#    history_player_stats,
#    context_game_stats,
#    mask_mode
#)

# player_time_dataset = PlayerSubDataset(time_dataset)
#print(len(time_dataset))

520


In [ ]:
np.random.seed(42)
game_indices = list(range(len(game_id_copy)))
np.random.shuffle(game_indices)
training_game_indices, testing_game_indices = game_indices[:int(0.7 * len(game_indices))], game_indices[int(0.7 * len(
    game_indices)):]
training_game_indices, validation_game_indices = training_game_indices[
                                                 int(0.2 * len(training_game_indices)):], training_game_indices[:int(
    0.2 * len(training_game_indices))]
training_games = game_id_copy[training_game_indices]

current_train_exp = history_player_stats[training_game_indices]
current_train_cum = current_player_stats[training_game_indices]
current_train_context = context_game_stats[training_game_indices]
current_validation_exp = history_player_stats[validation_game_indices]
current_validation_cum = current_player_stats[validation_game_indices]
current_validation_context = context_game_stats[validation_game_indices]
current_test_exp = history_player_stats[testing_game_indices]
current_test_cum = current_player_stats[testing_game_indices]
current_test_context = context_game_stats[testing_game_indices]

cumsum_train_exp = cumsum_history_player_stats[training_game_indices]
cumsum_train_cum = cumsum_current_player_stats[training_game_indices]
cumsum_validation_exp = cumsum_history_player_stats[validation_game_indices]
cumsum_validation_cum = cumsum_current_player_stats[validation_game_indices]
cumsum_test_exp = cumsum_history_player_stats[testing_game_indices]
cumsum_test_cum = cumsum_current_player_stats[testing_game_indices]

validation_games = game_id_copy[validation_game_indices]
testing_games = game_id_copy[testing_game_indices]

time_batch_size = 64
player_batch_size = 16

In [ ]:
def last_el_mask(out, mask):
    return out[torch.arange(out.shape[0]), mask - 1]


class OutputMode(Enum):
    TRAINING = 1
    TESTING = 2


class MaskMode(Enum):
    MAX_MINUTE_SIZE = 1
    FIXED = 2
    LAST_MINUTE = 3


In [ ]:
weighted_data = True

def get_datasets(mask_mode, cumsum_current, test=False):
    if cumsum_current == "cumsum":
        train_cum = cumsum_train_cum
        validation_cum = cumsum_validation_cum
        test_cum = cumsum_test_cum

        train_exp = cumsum_train_exp
        validation_exp = cumsum_validation_exp
        test_exp = cumsum_test_exp
    elif cumsum_current == "current":
        train_cum = current_train_cum
        validation_cum = current_validation_cum
        test_cum = current_test_cum

        train_exp = current_train_exp
        validation_exp = current_validation_exp
        test_exp = current_test_exp
    
    train_context = current_train_context
    validation_context = current_validation_context
    test_context = current_test_context

    if not test :

    
        time_train_dataset = MaskedPlayerDataset(training_game_indices, training_games, game_substitutions, train_cum,
                                                    train_exp, train_context, mask_mode)
        time_validation_dataset = MaskedPlayerDataset(validation_game_indices, validation_games, game_substitutions,
                                                        validation_cum, validation_exp, validation_context, mask_mode)
        time_test_dataset = None

        if weighted_data:
            subs = []
            items = []
            for i, data in enumerate(tqdm(time_train_dataset, desc='Calculating sample weights for training set')):
                item, _, _, _, _, _, time_sub, _, time_mask_length, _, _ = data
                if mask_mode == MaskMode.MAX_MINUTE_SIZE:
                    if torch.any(torch.flatten(time_sub) > 0):
                        subs.append(1)
                    else:
                        subs.append(0)
                elif mask_mode == MaskMode.LAST_MINUTE:
                    subs.append(time_sub[time_mask_length - 1].item())
            subs = np.array(subs).astype(int)
            classes, counts = np.unique(subs, return_counts=True)

            weight = 1. / counts
            train_samples_weight = weight[subs]
            train_sampler = WeightedRandomSampler(train_samples_weight, len(train_samples_weight),
                                                    generator=seed_new_generator())
            time_train_dataloader = DataLoader(time_train_dataset, batch_size=time_batch_size, sampler=train_sampler)
        else:
            time_train_dataloader = DataLoader(time_train_dataset, batch_size=time_batch_size, shuffle=True,
                                                generator=seed_new_generator())
        time_validation_dataloader = DataLoader(time_validation_dataset, batch_size=time_batch_size, shuffle=True,
                                                generator=seed_new_generator())
        time_test_dataloader = None
    else :
        time_train_dataset = None
        time_validation_dataset = None
        time_train_dataloader = None
        time_validation_dataloader = None
        time_test_dataset = MaskedPlayerDataset(testing_game_indices, testing_games, game_substitutions,
                                                        test_cum, test_exp, test_context, mask_mode)
        time_test_dataloader = DataLoader(time_test_dataset, batch_size=time_batch_size)
    
    if not test :

        player_train_dataset = PlayerSubDataset(time_train_dataset)
        player_validation_dataset = PlayerSubDataset(time_validation_dataset)
        player_train_dataloader = DataLoader(player_train_dataset, batch_size=player_batch_size, shuffle=True,
                                                generator=seed_new_generator())
        player_validation_dataloader = DataLoader(player_validation_dataset, batch_size=player_batch_size, shuffle=True,
                                                    generator=seed_new_generator())
        player_test_dataset = None
        player_test_dataloader = None
    else :
        player_train_dataset = None
        player_validation_dataset = None
        player_train_dataloader = None
        player_validation_dataloader = None
        player_test_dataset = player_test_dataset = PlayerSubDataset(time_test_dataset)
        player_test_dataloader = DataLoader(player_test_dataset, batch_size=player_batch_size)

    return time_train_dataloader, time_validation_dataloader, time_test_dataloader, player_train_dataloader, player_validation_dataloader, player_test_dataloader


In [ ]:
from torch.nn.modules.container import Sequential


class MaskedSoftmax(nn.Module):
    def __init__(self, dim=1):
        super(MaskedSoftmax, self).__init__()
        self.softmax = nn.Softmax(dim=dim)

    def forward(self, x, mask=None):
        x[mask == 0] = -float('inf')
        return self.softmax(x)


class MaskedLogSoftmax(nn.Module):
    def __init__(self, dim=1):
        super(MaskedLogSoftmax, self).__init__()
        self.softmax = nn.LogSoftmax(dim=dim)

    def forward(self, x, mask=None):
        x[mask == 0] = 1e-9
        return self.softmax(x)


class MaskedFeatureEncoder(nn.Module):
    def __init__(self, num_players, num_features, context_num_features, hidden_size, context_hidden_size, num_layers, dropout, device):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.context_hidden_size = context_hidden_size
        self.num_players = num_players
        self.num_features = num_features
        self.context_num_features = context_num_features
        self.dropout = dropout
        self.device = device

        self.player_expected_LSTM_layers = nn.ModuleList([nn.LSTM(self.num_players,
                                                                  self.hidden_size,
                                                                  self.num_layers,
                                                                  batch_first=True,
                                                                  dropout=self.dropout).to(self.device) for i in
                                                          range(self.num_features)])
        self.player_cumulated_LSTM_layers = nn.ModuleList([nn.LSTM(self.num_players,
                                                                   self.hidden_size,
                                                                   self.num_layers,
                                                                   batch_first=True,
                                                                   dropout=self.dropout).to(self.device) for i in
                                                           range(self.num_features)])
        self.context_LSTM_layer = nn.LSTM(self.context_num_features,
                                           self.context_hidden_size,
                                           self.num_layers,
                                           batch_first=True,
                                           dropout=self.dropout).to(self.device)
        self.bilinear_layers = nn.ModuleList([nn.Bilinear(self.hidden_size,
                                                          self.hidden_size,
                                                          self.hidden_size).to(self.device) for i in
                                              range(self.num_features)])

        self.exp_embedding_layers = nn.ModuleList(
            [nn.Linear(self.num_players, self.hidden_size) for i in range(self.num_features)]).to(self.device)
        self.cum_embedding_layers = nn.ModuleList(
            [nn.Linear(self.num_players, self.hidden_size) for i in range(self.num_features)]).to(self.device)
        self.context_embedding_layer = nn.Linear(self.context_num_features, self.context_hidden_size)

        self.attention_expected_layers = nn.ModuleList(
            [nn.MultiheadAttention(self.hidden_size, 2, batch_first=True) for i in range(self.num_features)]).to(
            self.device)
        self.attention_cumulated_layers = nn.ModuleList(
            [nn.MultiheadAttention(self.hidden_size, 2, batch_first=True) for i in range(self.num_features)]).to(
            self.device)
        self.attention_context_layer = nn.MultiheadAttention(self.context_hidden_size, 2, batch_first=True).to(self.device)

        self.leaky_relu = nn.LeakyReLU(0.01)

    def forward(self, exp, cum, context, mask_lengths):
        concat_outputs = []
        for i in range(self.num_features):
            expected_hidden_weight = torch.zeros(self.num_layers, exp.size(0), self.hidden_size).requires_grad_().to(
                self.device)
            expected_cell_weight = torch.zeros(self.num_layers, exp.size(0), self.hidden_size).requires_grad_().to(
                self.device)
            cumulated_hidden_weight = torch.zeros(self.num_layers, cum.size(0), self.hidden_size).requires_grad_().to(
                self.device)
            cumulated_cell_weight = torch.zeros(self.num_layers, cum.size(0), self.hidden_size).requires_grad_().to(
                self.device)

            packed_exp = nn.utils.rnn.pack_padded_sequence(exp[:, :, :, i], mask_lengths, batch_first=True,
                                                           enforce_sorted=False)
            packed_cum = nn.utils.rnn.pack_padded_sequence(cum[:, :, :, i], mask_lengths, batch_first=True,
                                                           enforce_sorted=False)

            packed_out_exp, (hn_exp, cn_exp) = self.player_expected_LSTM_layers[i](packed_exp, (
                expected_hidden_weight.detach(), expected_cell_weight.detach()))
            packed_out_cum, (hn_cum, cn_cum) = self.player_expected_LSTM_layers[i](packed_cum, (
                cumulated_hidden_weight.detach(), cumulated_cell_weight.detach()))
            out_exp, _ = nn.utils.rnn.pad_packed_sequence(packed_out_exp, batch_first=True)
            out_cum, _ = nn.utils.rnn.pad_packed_sequence(packed_out_cum, batch_first=True)

            attn_exp, _ = nn.utils.rnn.pad_packed_sequence(packed_exp, batch_first=True)
            attn_cum, _ = nn.utils.rnn.pad_packed_sequence(packed_cum, batch_first=True)
            attn_exp = self.exp_embedding_layers[i](attn_exp)
            attn_cum = self.exp_embedding_layers[i](attn_cum)
            attn_exp = self.leaky_relu(attn_exp)
            attn_cum = self.leaky_relu(attn_cum)
            attn_exp, _ = self.attention_expected_layers[i](attn_exp, attn_exp, attn_exp)
            attn_cum, _ = self.attention_cumulated_layers[i](attn_cum, attn_cum, attn_cum)

            out_exp = out_exp + attn_exp
            out_cum = out_cum + attn_cum

            out = self.bilinear_layers[i](out_exp, out_cum)
            concat_outputs.append(out)

        context_hidden_weight = torch.zeros(self.num_layers, context.size(0),
                                             self.context_hidden_size).requires_grad_().to(self.device)
        context_cell_weight = torch.zeros(self.num_layers, context.size(0),
                                           self.context_hidden_size).requires_grad_().to(self.device)
        packed_context = nn.utils.rnn.pack_padded_sequence(context, mask_lengths, batch_first=True, enforce_sorted=False)
        packed_out_context, (hn_context, cn_context) = self.context_LSTM_layer(packed_context, (context_hidden_weight.detach(), context_cell_weight.detach()))
        out_context, _ = nn.utils.rnn.pad_packed_sequence(packed_out_context, batch_first=True)
        attn_context, _ = nn.utils.rnn.pad_packed_sequence(packed_context, batch_first=True)

        attn_context = self.context_embedding_layer(attn_context)
        attn_context = self.leaky_relu(attn_context)
        attn_context, _ = self.attention_context_layer(attn_context, attn_context, attn_context)
        out_context = out_context + attn_context
        concat_outputs.append(out_context)

        out = torch.cat(concat_outputs, dim=2)
        out = self.leaky_relu(out)
        return out

    def freeze(self, boolean):
        for param in self.parameters():
            param.requires_grad = not boolean

    def get_first_weight(self):
        return list(self.parameters())[0]


class MaskedTimePredictionSubModel(nn.Module):
    def __init__(self, num_players, feature_size, hidden_size, nb_sub_batches, device) -> None:
        super().__init__()
        self.model_name = "TimePredictionSubModel"
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.num_players = num_players
        self.nb_sub_batches = nb_sub_batches
        self.device = device

        self.features_linear_layer = nn.Linear(self.feature_size, self.hidden_size).to(self.device)
        self.non_linear_activation_layer = nn.ReLU().to(self.device)

        self.output_layer = nn.Linear(self.hidden_size, self.nb_sub_batches).to(self.device)
        self.activation_layer = nn.Sigmoid().to(self.device)

    def forward(self, features):
        out = self.features_linear_layer(features)
        out = self.non_linear_activation_layer(out)

        out = self.output_layer(out)
        out = self.activation_layer(out)

        return out


class MaskedPlayerPredictionSubModel(nn.Module):
    def __init__(self, num_players, feature_size, hidden_size, output_mode, device) -> None:
        super().__init__()
        self.model_name = "TimePredictionSubModel"
        self.num_players = num_players
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.device = device

        self.features_linear_layer = nn.Linear(self.feature_size, self.num_players * 2).to(
            self.device)
        self.non_linear_activation_layer = nn.ReLU().to(self.device)

        self.bilinear_layer = nn.Bilinear(self.num_players * 2, self.num_players * 2, self.num_players * 2).to(
            self.device)

        self.player_in_prediction = nn.Linear(self.num_players * 2, self.num_players).to(self.device)
        self.player_out_prediction = nn.Linear(self.num_players * 2, self.num_players).to(self.device)
        self.output_mode = output_mode
        #self.activation_layer = MaskedSoftmax(dim=1).to(self.device)

    def forward(self, features, player_masks, time_mask_length):

        out = self.features_linear_layer(features)
        out = self.non_linear_activation_layer(out)
        out = last_el_mask(out, time_mask_length)

        player_out = self.player_out_prediction(out)
        player_in = self.player_in_prediction(out)

        if self.output_mode == OutputMode.TRAINING:
            player_out = MaskedLogSoftmax(dim=1)(player_out,
                                                 player_masks[:, player_on_field_index])

            player_in = MaskedLogSoftmax(dim=1)(player_in,
                                                player_masks[:, player_on_bench_index])
        elif self.output_mode == OutputMode.TESTING:
            player_out = MaskedSoftmax(dim=1)(player_out,
                                              player_masks[:, player_on_field_index])

            player_in = MaskedSoftmax(dim=1)(player_in,
                                             player_masks[:, player_on_bench_index])
        return player_out, player_in

    def get_first_weight(self):
        return list(self.parameters())[0]


class MaskedTimePlayerNeuralModel(nn.Module):
    def __init__(self, num_players, num_features, context_num_features, num_layers, num_minutes, hidden_size, context_hidden_size, nb_sub_batches, dropout, output_mode,
                 device) -> None:
        super().__init__()
        self.model_name = "TimePlayerNeuralModel"
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.context_hidden_size = context_hidden_size
        self.num_players = num_players
        self.num_features = num_features
        self.context_num_features = context_num_features
        self.num_minutes = num_minutes
        self.nb_sub_batches = nb_sub_batches
        self.dropout = dropout
        self.output_mode = output_mode
        self.device = device

        self.feature_size = self.hidden_size * self.num_features + self.context_hidden_size

        self.feature_encoder = MaskedFeatureEncoder(self.num_players,
                                                    self.num_features,
                                                    self.context_num_features,
                                                    self.hidden_size,
                                                    self.context_hidden_size,
                                                    self.num_layers,
                                                    self.dropout,
                                                    self.device).to(self.device)

        self.time_prediction_sub_model = MaskedTimePredictionSubModel(self.num_players,
                                                                      self.feature_size,
                                                                      self.hidden_size,
                                                                      self.nb_sub_batches,
                                                                      self.device).to(self.device)

        self.player_prediction_sub_model = MaskedPlayerPredictionSubModel(self.num_players,
                                                                          self.feature_size,
                                                                          self.hidden_size,
                                                                          self.output_mode,
                                                                          self.device).to(self.device)

    def forward(self, exp, cum, context, time_sub, player_mask, time_mask, time_mask_length):
        encoded_features = self.feature_encoder(exp, cum, context, time_mask_length)

        player_out = torch.zeros((exp.shape[0], nb_sub_batches, self.num_players)).to(device)
        player_in = torch.zeros((exp.shape[0], nb_sub_batches, self.num_players)).to(device)

        time_pred = self.time_prediction_sub_model(encoded_features)
        out_placeholder = torch.zeros(time_pred.shape[0], self.num_minutes, self.nb_sub_batches).to(device)
        out_placeholder[0:time_pred.shape[0], 0:time_pred.shape[1], 0:time_pred.shape[2]] = time_pred[:, :, :]
        time_pred = torch.masked_fill(out_placeholder, time_mask == 0, 1e-5)
        time_out = torch.unsqueeze(time_pred, dim=-1)

        minute_player_mask = last_el_mask(player_mask, time_mask_length)
        for i in range(self.nb_sub_batches) :
            player_out[:, i], player_in[:, i] = self.player_prediction_sub_model(encoded_features,
                                                                                            minute_player_mask[:, i],
                                                                                            time_mask_length)
        return time_out, player_out, player_in

    def freeze(self, boolean):
        self.feature_encoder.freeze(boolean)

    def get_feature_encoder_first_weight(self):
        return self.feature_encoder.get_first_weight()

    def get_player_prediction_first_weight(self):
        return self.player_prediction_sub_model.get_first_weight()

    def player_prediction_parameters(self):
        return self.player_prediction_sub_model.parameters()

    def set_output_mode(self, mode):
        assert (type(mode) == OutputMode)
        self.player_prediction_sub_model.output_mode = mode

In [ ]:
class CustomBCELoss(nn.Module):
    def __init__(self, scale=2):
        super().__init__()
        self.scale = scale

    def forward(self, output, target, mask=None):
        #game_index = 1
        #print(output[game_index])
        #print(target[game_index])
        #print(mask[game_index])
        ones = (target == 1).nonzero()
        seq_size = target.shape[1]
        output_indices = torch.arange(output.shape[1]).to(device)
        num_substitutions = torch.zeros(output.shape[0]).to(device)
        loss = torch.full((target.shape[0], target.shape[1]), 1e-5).to(device)
        loss = loss - 1
        for index in ones:
            loss[index[0]] = loss[index[0]] - (1 + torch.abs(index[1] - output_indices) / seq_size)
            num_substitutions[index[0]] += 1

        #print(loss[game_index])
        loss = torch.transpose(loss, 1, 0)
        num_substitutions[num_substitutions == 0] = 1
        #print(loss[game_index])
        loss = torch.div(loss, num_substitutions)
        loss = torch.transpose(loss, 1, 0)
        #print(loss[game_index])
        loss = torch.unsqueeze(loss, dim=2)
        loss = loss * (-1)
        loss_min, loss_max = loss.min(), loss.max()
        new_min, new_max = 1, self.scale
        if loss_min < loss_max:
            loss = (loss - loss_min) / (loss_max - loss_min) * (new_max - new_min) + new_min
        #print(loss[game_index])
        if mask != None:
            mask = mask.to(device)
            loss = loss * mask
        #print(loss[game_index])
        BCELoss = torch.nn.BCELoss(reduction='none')(output, target)
        loss = loss * BCELoss
        #print(loss[game_index])
        loss = torch.sum(loss)
        #print(loss)
        #print(fdsg)
        #loss = torch.mean(loss)
        return loss


class LastMinuteCustomBCELoss(nn.Module):
    def __init__(self, scale=2):
        super().__init__()
        self.scale = scale

    def norm(self, dist, minute):
        max = torch.max(torch.abs(self.num_minutes - minute), minute + 1)
        min = 1
        out = 1 + (dist - 1) * (self.scale - min) / (max - min)
        return out

    def forward(self, output, target, minutes):
        ones = (target == 1).nonzero()
        num_substitutions = torch.zeros(target.shape[0]).to(device)
        total_distance = torch.ones(target.shape[0]).to(device)
        loss = nn.BCELoss(reduction="none")(output, target[torch.arange(target.shape[0]), minutes])
        loss = torch.squeeze(loss)
        v = False
        for index in ones:
            total_distance[index[0]] += 1 + torch.abs(index[1] - minutes[index[0]])
            num_substitutions[index[0]] += 1
            #bce = nn.BCELoss(reduction="none")(output[index[0]], target[index[0], minutes[index[0]]])
            if v:
                print('minute')
                print(minutes[index[0]])
                print('minute sub')
                print(index[1])
                print('total_distance')
                print(total_distance[index[0]])
            if v:
                print(loss[index[0]])
            v = False

        num_substitutions[num_substitutions == 0] = 1
        loss *= total_distance
        loss /= num_substitutions
        loss = torch.sum(loss)
        return loss

In [ ]:
eps = 1e-10


def train_one_epoch_time_prediction(epoch_number, tb_writer, training_dataloader, training_config):
    running_loss = 0.
    last_loss = 0.

    loss_f = CustomBCELoss(scale=training_config.scale)

    #training_dataloader.sampler.generator.manual_seed(42)
    t = tqdm(training_dataloader,
             desc=f'Training Time Prediction... Epoch : {epoch_number + 1} - Dataset size : {len(training_dataloader)}',
             bar_format="Training Loss : {postfix[0]:.5f}, Iteration : {postfix[1]} / {postfix[2]} - {percentage:.0f}%|{bar}| [{elapsed}<{remaining}]",
             postfix=[last_loss, 0, len(training_dataloader)])
    for i, data in enumerate(t):
        optimizer.zero_grad()
        _, _, minutes, exp, cum, context, time_sub, time_mask, time_mask_length, _, player_mask = data
        time_out, _, _ = model(exp, cum, context, time_sub, player_mask, time_mask, time_mask_length)
        loss = loss_f(time_out[:, :, 0], time_sub[:, :, 0], time_mask)
        for i in range(nb_sub_batches - 1):
            loss += loss_f(time_out[:, :, i + 1], time_sub[:, :, i + 1], time_mask)
        loss.backward()
        optimizer.step()
        last_loss = loss.item()
        running_loss += last_loss
        tb_x = epoch_number * len(training_dataloader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        t.postfix = [last_loss, i, len(training_dataloader)]
        t.update()
    return running_loss


def train_one_epoch_player_prediction(epoch_number, tb_writer, training_dataloader, training_config):
    running_loss = 0.
    last_loss = 0.
    loss_f = nn.NLLLoss()
    #training_dataloader.sampler.generator.manual_seed(42)
    t = tqdm(training_dataloader,
             desc=f'Training Player Prediction... Epoch : {epoch_number + 1} - Dataset size : {len(training_dataloader)}',
             bar_format="Training Loss : {postfix[0]:.5f}, Iteration : {postfix[1]} / {postfix[2]} - {percentage:.0f}%|{bar}| [{elapsed}<{remaining}]",
             postfix=[last_loss, 0, len(training_dataloader)])
    for i, data in enumerate(t):
        optimizer.zero_grad()
        _, _, _, exp, cum, context, time_sub, time_mask, time_mask_length, player_sub, player_mask = data
        true_player_out = player_sub[:, :, player_out_index]
        true_player_in = player_sub[:, :, player_in_index]
        _, player_out, player_in = model(exp, cum, context, time_sub, player_mask, time_mask, time_mask_length)
        player_out_loss = loss_f(player_out[:, 0, :], true_player_out[:, 0])
        player_in_loss = loss_f(player_in[:, 0, :], true_player_in[:, 0])
        for i in range(nb_sub_batches - 1):
            player_out_loss += loss_f(player_out[:, i + 1, :], true_player_out[:, i + 1])
            player_in_loss += loss_f(player_in[:, i + 1, :], true_player_in[:, i + 1])
        loss = player_out_loss + player_in_loss
        loss.backward()
        optimizer.step()
        last_loss = loss.item()
        running_loss += last_loss
        tb_x = epoch_number * len(training_dataloader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        t.postfix = [last_loss, i, len(training_dataloader)]
        t.update()
    return running_loss


def eval_one_epoch_time_prediction(validation_dataloader, training_config):
    running_vloss = 0.
    loss_f = CustomBCELoss(scale=training_config.scale)

    t = tqdm(validation_dataloader,
             desc=f'Evaluating Time Prediction... Epoch : {epoch_number + 1} - Loss : {running_vloss} - Dataset size : {len(validation_dataloader)}',
             bar_format="Validation Loss : {postfix[0]:.5f}, Iteration : {postfix[1]} / {postfix[2]} - {percentage:.0f}%|{bar}| [{elapsed}<{remaining}]",
             postfix=[running_vloss, 0, len(validation_dataloader)])

    for i, data in enumerate(t):
        _, _, minutes, exp, cum, context, time_sub, time_mask, time_mask_length, player_sub, player_mask = data
        time_out, _, _ = model(exp, cum, context, time_sub, player_mask, time_mask, time_mask_length)
        vloss = loss_f(time_out[:, :, 0], time_sub[:, :, 0], time_mask)
        for i in range(nb_sub_batches - 1):
            vloss += loss_f(time_out[:, :, i + 1], time_sub[:, :, i + 1], time_mask)
        last_vloss = vloss.item()
        running_vloss += last_vloss
        t.postfix = [last_vloss, i, len(validation_dataloader)]
        t.update()
    return running_vloss


def eval_one_epoch_player_prediction(validation_dataloader, training_config):
    running_vloss = 0.
    loss_f = nn.NLLLoss()
    t = tqdm(validation_dataloader,
             desc=f'Evaluating Time Prediction... Epoch : {epoch_number + 1} - Loss : {running_vloss} - Dataset size : {len(validation_dataloader)}',
             bar_format="Validation Loss : {postfix[0]:.5f}, Iteration : {postfix[1]} / {postfix[2]} - {percentage:.0f}%|{bar}| [{elapsed}<{remaining}]",
             postfix=[running_vloss, 0, len(validation_dataloader)])

    for i, data in enumerate(t):
        _, _, _, exp, cum, context, time_sub, time_mask, time_mask_length, player_sub, player_mask = data
        _, player_out, player_in = model(exp, cum, context, time_sub, player_mask, time_mask, time_mask_length)

        true_player_out = player_sub[:, :, player_out_index]
        true_player_in = player_sub[:, :, player_in_index]

        player_out_loss = loss_f(player_out[:, 0, :], true_player_out[:, 0])
        player_in_loss = loss_f(player_in[:, 0, :], true_player_in[:, 0])
        for i in range(nb_sub_batches - 1):
            player_out_loss += loss_f(player_out[:, i + 1, :], true_player_out[:, i + 1])
            player_in_loss += loss_f(player_in[:, i + 1, :], true_player_in[:, i + 1])
        vloss = player_out_loss + player_in_loss
        last_vloss = vloss.item()
        running_vloss += last_vloss
        t.postfix = [last_vloss, i, len(validation_dataloader)]
        t.update()
    return running_vloss


In [ ]:
time_train_dataloader, time_validation_dataloader, time_test_dataloader, player_train_dataloader, player_validation_dataloader, player_test_dataloader = get_datasets(MaskMode.MAX_MINUTE_SIZE, "current", test=False)

Extracting player substitutions:   0%|          | 0/292 [00:00<?, ?it/s]

Extracting player substitutions:   0%|          | 0/72 [00:00<?, ?it/s]

Calculating sample weights for training set:   0%|          | 0/28324 [00:00<?, ?it/s]

Building player sub dataset:   0%|          | 0/28324 [00:00<?, ?it/s]

In [ ]:
base_model_name = 'player_neural_model'
time_epoch = 2
player_epoch = 10
hidden_sizes = [4, 8, 16, 32]
context_hidden_sizes = [8, 16, 32]
learning_rates = [0.001]
steps = [0.1]
mask_modes = [MaskMode.MAX_MINUTE_SIZE]
scales = [10]
nums_layers = [2, 3, 4]
loss_function = "CustomBCELoss"
def initialize_training_configs():
    training_configs = {}
    training_configs['list'] = []
    for cumsum_current in ["current"]:
        for hidden_size in hidden_sizes:
            for context_hidden_size in context_hidden_sizes:
                for num_layers in nums_layers:
                    for scale in scales:
                        for learning_rate in learning_rates:
                            for step in steps:
                                model_code = f'd{data_name}-h{hidden_size}-ch{context_hidden_size}-n{num_layers}-sc{scale}-lr{learning_rate}-s{step}-do{dropout}'
                                training_config = {}
                                training_config['config'] = dotdict({
                                    "model_name": f'{data_name} - hidden {hidden_size} - context hidden {context_hidden_size} - #layers {num_layers}',
                                    "model_code" : f'd{data_name}-h{hidden_size}-ch{context_hidden_size}-n{num_layers}-sc{scale}-lr{learning_rate}-s{step}-do{dropout}',
                                    "cumsum_current": cumsum_current,
                                    "time_epochs": time_epoch,
                                    "player_epochs": player_epoch,
                                    "loss_function": loss_function,
                                    "scale": scale,
                                    "hidden_size": hidden_size,
                                    "context_hidden_size": context_hidden_size,
                                    "num_layers": num_layers,
                                    "dropout": dropout,
                                    "learning_rate": learning_rate,
                                    "step": step
                                })
                                training_config['results'] = {}
                                
                                model_path = os.path.join(modelfolder, model_code)
                                result_path = os.path.join(datafolder, f"{training_config['config'].model_name}_pred_player_sub")
                                if os.path.exists(model_path):
                                    training_config['model_path'] = model_path
                                    training_config['train'] = False
                                else :
                                    training_config['model_path'] = ''
                                    training_config['train'] = True
                                if os.path.exists(result_path) :
                                    training_config['result_path'] = result_path
                                    training_config['test'] = False
                                else :
                                    training_config['result_path'] = ''
                                    training_config['test'] = True
                                training_configs[model_code] = dotdict(training_config)
                                training_configs['list'].append(model_code)
    training_configs = dotdict(training_configs)
    return training_configs

In [ ]:
base_model_name = 'player_neural_model'
time_epoch = 2
player_epoch = 10
hidden_sizes = [4, 8, 16, 32]
context_hidden_sizes = [8, 16, 32]
learning_rates = [0.001]
steps = [0.1]
mask_modes = [MaskMode.MAX_MINUTE_SIZE]
scales = [10]
nums_layers = [2, 3, 4]
loss_function = "CustomBCELoss"
dropout = 0.2
data_name = "Barcelona" if dataset_version == StatsBombDataset.BARCELONA else "Women"

modelfolder = os.path.join(base_path, "model_weights/")
resultsfolder = os.path.join(base_path, "results/")
training_config_file = os.path.join(resultsfolder, f"training_configs_{data_name}_{base_model_name}.json")

if not os.path.exists(training_config_file) :
    with open(training_config_file, 'r') as fp:
        training_configs = json.load(fp)
else :
    training_configs = initialize_training_configs()
    with open(training_config_file, 'w') as fp:
        json.dump(training_configs, fp)

print(training_configs)

In [ ]:

train = True
current_mask_mode = None
if train:
    training_config_list = tqdm(enumerate(training_configs['list']))
    for i, model_code in training_config_list:
        training_config = dotdict(training_configs[model_code]['config'])
        model_name = training_config.model_name
        print(f"Model : {model_name}")
        train_model = training_configs[model_code]['train']
        model_path = os.path.join(modelfolder, model_code)
        if train_model : 
            
            torch.manual_seed(seed)
            # if current_mask_mode != training_config.mask_mode:
            #     current_mask_mode = training_config.mask_mode
            #     time_train_dataloader, time_validation_dataloader, _, player_train_dataloader, player_validation_dataloader, _ = get_datasets(
            #         current_mask_mode, training_config.cumsum_current, test=False)
            time_dataset = time_train_dataloader.dataset
            model = MaskedTimePlayerNeuralModel(num_players=time_dataset.num_players,
                                                num_features=time_dataset.num_features,
                                                context_num_features=time_dataset.context_num_features,
                                                num_layers=training_config.num_layers,
                                                num_minutes=time_dataset.num_minutes,
                                                hidden_size=training_config.hidden_size,
                                                context_hidden_size=training_config.context_hidden_size,
                                                dropout=training_config.dropout,
                                                nb_sub_batches=nb_sub_batches,
                                                output_mode=OutputMode.TRAINING,
                                                device=device).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=training_config.learning_rate, betas=[0.8, 0.999],
                                         weight_decay=1e-3)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, training_config.step)
            loss_value = 0
            total_steps = 0

            writer_path = os.path.join(base_path, f'runs/' + model_name)
            if os.path.exists(writer_path):
                shutil.rmtree(writer_path)
            writer = SummaryWriter(writer_path)
            print(f'Training {model_name}')
            EPOCHS = training_config.time_epochs
            epoch_number = 0

            best_time_vloss = 1_000_000.
            best_time_weigths = model.state_dict()
            best_player_vloss = 1_000_000.
            best_player_weigths = model.state_dict()

            train_time = True
            if train_time:
                while epoch_number < EPOCHS:
                    print('### TIME PREDICTION TRAINING ###')
                    print('EPOCH {}:'.format(epoch_number + 1))

                    #Time prediction training
                    model.train(True)
                    model.freeze(False)
                    time_running_loss = train_one_epoch_time_prediction(epoch_number, writer, time_train_dataloader,
                                                                        training_config)
                    torch.cuda.empty_cache()
                    model.train(False)
                    time_running_vloss = eval_one_epoch_time_prediction(time_validation_dataloader, training_config)

                    time_avg_loss = time_running_loss / time_batch_size
                    time_avg_vloss = time_running_vloss / time_batch_size

                    print(
                        f'Epoch trained : Average Time Training Loss {time_avg_loss} - Average Time Validation Loss {time_avg_vloss}')

                    writer.add_scalars('Time Training vs. Time Validation Loss',
                                       {'Training': time_avg_loss, 'Validation': time_avg_vloss},
                                       epoch_number + 1)
                    writer.flush()
                    epoch_model_path = os.path.join(modelfolder, f'{model_name}-{epoch_number}')
                    
                    if time_avg_vloss < best_time_vloss:
                        print('better vloss')
                        best_time_vloss = time_avg_vloss
                        best_time_weigths = model.state_dict()
                    else:
                        print('not better vloss')
                        model.load_state_dict(best_time_weigths)

                    scheduler.step()
                    epoch_number += 1
                torch.save(best_time_weigths, model_path)
            #else:
            #    model.load_state_dict(torch.load(model_path))

            optimizer = torch.optim.Adam(model.player_prediction_sub_model.parameters(), lr=0.001,
                                         betas=[0.9, 0.999])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.9)

            EPOCHS = training_config.player_epochs
            epoch_number = 0
            while epoch_number < EPOCHS:
                #Player prediction training
                print('### PLAYER PREDICTION TRAINING ###')
                print('EPOCH {}:'.format(epoch_number + 1))

                model.train(True)
                model.freeze(True)
                v = False
                if v:
                    for name, param in model.named_parameters():
                        if name in ["player_prediction_sub_model.player_out_prediction.weight",
                                    "feature_encoder.attention_cumulated_layers.3.concat_linear.weight"]:
                            print('name: ', name)
                            print('param.shape: ', param.shape)
                            print('param.requires_grad: ', param.requires_grad)
                            print(param)
                            print('=====')
                player_running_loss = train_one_epoch_player_prediction(epoch_number, writer,
                                                                        player_train_dataloader, training_config)

                if v:
                    for name, param in model.named_parameters():
                        if name in ["player_prediction_sub_model.player_out_prediction.weight",
                                    "feature_encoder.attention_cumulated_layers.3.concat_linear.weight"]:
                            print('name: ', name)
                            print('param.shape: ', param.shape)
                            print('param.requires_grad: ', param.requires_grad)
                            print('=====')

                torch.cuda.empty_cache()

                model.train(False)
                player_running_vloss = eval_one_epoch_player_prediction(player_validation_dataloader,
                                                                        training_config)

                player_avg_loss = player_running_loss / player_batch_size
                player_avg_vloss = player_running_vloss / player_batch_size

                print(
                    f'Epoch trained : Average Player Training Loss {player_avg_loss} - Average Player Validation Loss {player_avg_vloss}')

                writer.add_scalars('Player Training vs. Player Validation Loss',
                                   {'Training': player_avg_loss, 'Validation': player_avg_vloss},
                                   epoch_number + 1)
                writer.flush()

                if player_avg_vloss < best_player_vloss:
                    best_player_vloss = player_avg_vloss
                    best_player_weigths = model.state_dict()
                else:
                    model.load_state_dict(best_player_weigths)
                scheduler.step()
                epoch_number += 1
            torch.save(best_player_weigths, model_path)
            del model
        else:
            print('Model already trained, skipping to next')

            

In [ ]:
_, time_validation_dataloader, time_test_dataloader, _, player_validation_dataloader, player_test_dataloader = get_datasets(MaskMode.MAX_MINUTE_SIZE, "current", test=True)
model_predictions = {}
model_stats = {}
model_names = []

Extracting player substitutions:   0%|          | 0/156 [00:00<?, ?it/s]

Building player sub dataset:   0%|          | 0/15132 [00:00<?, ?it/s]

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

def extract_true_data(dataset):
    true_time_sub = dataset.time_substitutions.cpu().numpy()
    true_player_sub = dataset.player_substitutions.cpu().numpy()
    true_game_ids = dataset.game_id_list

    return true_time_sub, true_player_sub, true_game_ids

def extract_baseline_train_data(dataset):
    train_time_sub = dataset.time_substitutions.cpu().numpy()
    return train_time_sub

def test_model(model, dataset, mask_mode):
    with torch.no_grad():
        model.eval()
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True, generator=seed_new_generator())
        t = tqdm(dataloader)

        pred_time_sub = np.zeros((dataset.num_games, dataset.num_minutes, nb_sub_batches))
        pred_player_sub = np.zeros((dataset.num_games, dataset.num_minutes, nb_sub_batches, 2))
        v = False

        for j, data in enumerate(t):
            item, game_index, minute, exp, cum, context, time_sub, time_mask, time_mask_length, player_sub, player_mask = data

            time_out, player_out, player_in = model(exp, cum, context, time_sub, player_mask, time_mask, time_mask_length)
            minute_player_mask = player_mask[torch.arange(player_mask.shape[0]), time_mask_length - 1]

            time_out = time_out.detach().cpu().numpy()
            player_out = player_out.detach().cpu().numpy()
            player_in = player_in.detach().cpu().numpy()

            for i, mask_length in enumerate(time_mask_length):
                pred_time_sub[game_index[i], minute[i]] = np.squeeze(time_out[i][mask_length - 1])
                for j in range(nb_sub_batches) :
                    pred_player_sub[game_index[i], minute[i], j, player_out_index] = np.argmax(player_out[i, j])
                    pred_player_sub[game_index[i], minute[i], j, player_in_index] = np.argmax(player_in[i, j])
                if v:
                    print(minute[i])
                    print(pred_time_sub[game_index[i]])
                #v = False

        return pred_time_sub, pred_player_sub


In [ ]:
!nvidia-smi

Fri Aug 12 12:31:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    35W / 250W |   1533MiB / 16280MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:


def get_model_files_from_training_configs():
    model_files = set()
    for training_config in training_configs:
        print(f'Base name : {training_config.model_name}')
        for file in os.listdir(modelfolder):
            if file.startswith(f'{training_config.model_name}'):
                print(f'Model found {file}')
                model_files.add(file)
    return model_files


test = True
time_test_dataset = time_test_dataloader.dataset
if test:
    true_time_sub, true_player_sub, true_game_ids = extract_true_data(time_test_dataset)
    
    for model_code in training_configs['list']:
        training_config = dotdict(training_configs[model_code]['config'])
        print(training_config)
       
        model_name = training_config.model_name
        model_path = training_configs[model_code]['model_path']
        if model_path != '': 
            result_path = training_configs[model_code]['result_path']
            print(model_name)

            time_prediction_path = os.path.join(resultsfolder, f'{model_code}_pred_time_sub.pt')
            player_prediction_path = os.path.join(resultsfolder, f'{model_code}_pred_player_sub.pt')

            if not os.path.exists(time_prediction_path):
                print("Computing predictions")
                model_names.append(model_name)
                model = MaskedTimePlayerNeuralModel(num_players=time_test_dataset.num_players,
                                                    num_features=time_test_dataset.num_features,
                                                    num_layers=training_config.num_layers,
                                                    context_num_features=time_test_dataset.context_num_features,
                                                    context_hidden_size=training_config.context_hidden_size,
                                                    num_minutes=time_test_dataset.num_minutes,
                                                    hidden_size=training_config.hidden_size,
                                                    dropout=training_config.dropout,
                                                    output_mode=OutputMode.TESTING,
                                                    nb_sub_batches=nb_sub_batches,
                                                    device=device).to(device)
                model.eval()
                model.load_state_dict(torch.load(model_path))

                #if training_config.mask_mode == MaskMode.LAST_MINUTE :
                result = test_model(model, time_test_dataloader.dataset, training_config.mask_mode)
                pred_time_sub = result[0]
                pred_player_sub = result[1]

                torch.save(pred_time_sub, time_prediction_path)
                torch.save(pred_player_sub, player_prediction_path)

                training_configs[model_code]['result_path'] = time_prediction_path
            else :
                print("Predictions already computed")
                pred_time_sub = torch.load(time_prediction_path)
                pred_player_sub = torch.load(player_prediction_path)
            model_prediction = {
                'pred_time_sub': pred_time_sub,
                'pred_player_sub': pred_player_sub
            }
            model_prediction = dotdict(model_prediction)
            model_predictions[model_code] = model_prediction

{'model_name': 'Barcelona - hidden 4 - context hidden 8 - #layers 2', 'model_code': 'dBarcelona-h4-ch8-n2-sc10-lr0.001-s0.1-do0.2', 'cumsum_current': 'current', 'time_epochs': 2, 'player_epochs': 10, 'loss_function': 'CustomBCELoss', 'scale': 10, 'hidden_size': 4, 'context_hidden_size': 8, 'num_layers': 2, 'dropout': 0.2, 'learning_rate': 0.001, 'step': 0.1}
Barcelona - hidden 4 - context hidden 8 - #layers 2
Predictions already computed
{'model_name': 'Barcelona - hidden 4 - context hidden 8 - #layers 3', 'model_code': 'dBarcelona-h4-ch8-n3-sc10-lr0.001-s0.1-do0.2', 'cumsum_current': 'current', 'time_epochs': 2, 'player_epochs': 10, 'loss_function': 'CustomBCELoss', 'scale': 10, 'hidden_size': 4, 'context_hidden_size': 8, 'num_layers': 3, 'dropout': 0.2, 'learning_rate': 0.001, 'step': 0.1}
Barcelona - hidden 4 - context hidden 8 - #layers 3
Predictions already computed
{'model_name': 'Barcelona - hidden 4 - context hidden 8 - #layers 4', 'model_code': 'dBarcelona-h4-ch8-n4-sc10-lr0.0

In [ ]:
import sys

np.set_printoptions(threshold=sys.maxsize)


def dtw_roc_curve(true, pred, thresholds, window=1):
    true = np.squeeze(true)
    clone_pred = np.squeeze(pred).copy()

    fpr = np.zeros((thresholds.size,))
    tpr = np.zeros((thresholds.size,))
    precision = np.zeros((thresholds.size,))
    recall = np.zeros((thresholds.size,))

    for i, threshold in tqdm(enumerate(thresholds), desc='Computing Dynamic Time TPR and FPR', total=len(thresholds)):
        clone_pred[pred < threshold] = 0
        clone_pred[pred >= threshold] = 1

        tps, tns, fps, fns = 0, 0, 0, 0
        for j in range(true.shape[0]):
            tp, tn, fp, fn = dtw_classification(true[j], clone_pred[j], window)
            tps += tp
            tns += tn
            fps += fp
            fns += fn
        tpr[i] = tps / (tps + fns)
        fpr[i] = fps / (fps + tns)
        precision[i] = tps / (tps + fps)
        recall[i] = tps / (tps + fns)
    return fpr, tpr, precision, recall


def dtw_classification(true, pred, window=1):
    def window_coordinates(index):
        arange = np.arange(start=index - window, stop=index + window + 1)
        arange[arange < 0] = 0
        arange[arange > true.shape[0] - 1] = true.shape[0] - 1
        return arange

    tp = np.zeros((true.shape[0],))
    tn = np.zeros((true.shape[0],))
    fn = np.zeros((true.shape[0],))
    fp = np.zeros((true.shape[0],))

    subs = np.argwhere(true > 0)
    sub_windows = np.array(list(map(window_coordinates, subs)))
    pred_sub_in_sub_windows = np.any(pred[sub_windows], axis=1)
    not_pred_sub_in_sub_windows = np.logical_not(pred_sub_in_sub_windows)
    true_pred_sub = subs[pred_sub_in_sub_windows]
    not_pred_sub = subs[not_pred_sub_in_sub_windows]
    tp[true_pred_sub] = 1
    fn[not_pred_sub] = 1

    non_sub = np.squeeze(np.argwhere(true == 0), axis=-1)
    true_non_sub = np.squeeze(non_sub[np.argwhere(true[non_sub] == pred[non_sub])])
    tn[true_non_sub] = 1

    false_non_sub = np.squeeze(non_sub[np.argwhere(true[non_sub] != pred[non_sub])], axis=-1)
    non_sub_in_sub_windows = np.isin(false_non_sub, sub_windows)
    tp_false_non_sub = false_non_sub[non_sub_in_sub_windows]
    fp_false_non_sub = false_non_sub[np.logical_not(non_sub_in_sub_windows)]

    tp[tp_false_non_sub] = 1
    fp[fp_false_non_sub] = 1

    #print(f'tp   : {tp}')
    #print(f'tn   : {tn}')
    #print(f'fn   : {fn}')
    #print(f'fp   : {fp}')

    #print(f'total: {tp + tn + fp + fn}')

    tp = np.sum(tp)
    tn = np.sum(tn)
    fp = np.sum(fp)
    fn = np.sum(fn)

    assert ((tp + tn + fp + fn) == true.shape[0])

    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    return tp, tn, fp, fn

In [ ]:
print(true_time_sub.shape)

train_time_sub = extract_baseline_train_data(time_train_dataloader.dataset)
by_batch_true_time_sub = np.squeeze(train_time_sub)
print(by_batch_true_time_sub.shape)
one_batch_true_time_sub = np.squeeze(np.sum(by_batch_true_time_sub, axis=2))
print(one_batch_true_time_sub.shape)

by_batch_baseline = np.mean(by_batch_true_time_sub, axis=0)
print(by_batch_baseline.shape)
by_batch_baseline = np.broadcast_to(by_batch_baseline, (true_time_sub.shape[0], true_time_sub.shape[1], 3))
print(by_batch_baseline.shape)
model_predictions['by_batch_mean_baseline'] = {'pred_time_sub' : by_batch_baseline}
model_names.append('by_batch_mean_baseline')

one_batch_baseline = np.mean(one_batch_true_time_sub, axis=0)
print(one_batch_baseline.shape)
one_batch_baseline = np.expand_dims(one_batch_baseline, axis=-1)
print(one_batch_baseline.shape)
one_batch_baseline = np.broadcast_to(one_batch_baseline, (true_time_sub.shape[0], true_time_sub.shape[1], 3))
model_predictions['one_batch_mean_baseline'] = {'pred_time_sub' : one_batch_baseline}
model_names.append('one_batch_mean_baseline')

naive_baseline = np.zeros((true_time_sub.shape[0], true_time_sub.shape[1], 3))
naive_baseline[:, 45, 0] = 1
naive_baseline[:, 60, 1] = 1
naive_baseline[:, 77, 2] = 1
model_predictions['naive_baseline'] = {'pred_time_sub' : naive_baseline}
model_names.append('naive_baseline')


(156, 99, 3, 1)
(292, 99, 3)
(292, 99)
(99, 3)
(156, 99, 3)
(99,)
(99, 1)


In [ ]:
from sklearn.metrics import f1_score, auc

print(true_time_sub.shape)
true_time_sub = np.squeeze(true_time_sub)
flatten_true_time_sub = true_time_sub.flatten()
print(flatten_true_time_sub.shape)

stats_columns = ['model_name','model_code','batch','auc_score', 'brier_score', 'gmeans', 'gmeans_threshold', 'J', 'J_threshold', 'fscore', 'fscore_threshold']
model_stats = []

print(model_predictions.keys())

for i in range(nb_sub_batches):
    print(f"batch {i}")
    true_batch_time_sub = true_time_sub[:, :, i]
    flatten_true_batch_time_sub = true_batch_time_sub.flatten()

    fig = go.Figure()

    for model_code in training_configs['list'] :
        model_name = training_configs[model_code]['config']['model_name']
        if model_code in model_predictions.keys() :
            print(model_name)
            batch_pred_sub = model_predictions[model_code]['pred_time_sub']
            batch_pred_sub = batch_pred_sub[:, :, i]

            flatten_batch_pred_sub = batch_pred_sub.flatten()
            fpr, tpr, roc_thresholds = roc_curve(flatten_true_batch_time_sub, flatten_batch_pred_sub, drop_intermediate=True)
            precision, recall, pr_thresholds = precision_recall_curve(flatten_true_batch_time_sub, flatten_batch_pred_sub)
            #fpr, tpr, precision, recall = dtw_roc_curve(true_batch_time_sub, batch_pred_sub, roc_thresholds, window=2)
            auc_score = auc(fpr, tpr)
            brier_score = brier_score_loss(flatten_true_batch_time_sub, flatten_batch_pred_sub)

            gmeans = np.sqrt(tpr * (1 - fpr))
            best_gmeans = np.nanmax(gmeans)
            best_gmeans_threshold_index = np.nanargmax(gmeans)
            best_gmeans_threshold = roc_thresholds[best_gmeans_threshold_index]

            J = tpr - fpr
            best_J = np.nanmax(J)
            best_J_threshold_index = np.nanargmax(J)
            best_J_threshold = roc_thresholds[best_J_threshold_index]

            fscore = (2 * precision * recall) / (precision + recall)
            best_fscore = np.nanmax(fscore)
            print(best_fscore)
            best_fscore_threshold_index = np.nanargmax(fscore)
            best_fscore_threshold = pr_thresholds[best_fscore_threshold_index]

            stats = [model_name, model_code, i, auc_score, brier_score, best_gmeans, best_gmeans_threshold, best_J, best_J_threshold, best_fscore, best_fscore_threshold]
            model_stats.append(stats)
            auc_formatted = "{:.2f}".format(auc_score)
            fig.add_trace(
                go.Scatter(
                    name=f'{model_name} - {auc_formatted}',
                    mode="lines",
                    x=fpr,
                    y=tpr,
                )
            )


    for baseline_name in ['by_batch_mean_baseline', 'one_batch_mean_baseline', 'naive_baseline'] :
        batch_pred_sub = model_predictions[baseline_name]['pred_time_sub']
        batch_pred_sub = batch_pred_sub[:, :, i]

        flatten_batch_pred_sub = batch_pred_sub.flatten()
        fpr, tpr, roc_thresholds = roc_curve(flatten_true_batch_time_sub, flatten_batch_pred_sub, drop_intermediate=True)
        precision, recall, pr_thresholds = precision_recall_curve(flatten_true_batch_time_sub, flatten_batch_pred_sub)
        #fpr, tpr, precision, recall = dtw_roc_curve(true_batch_time_sub, batch_pred_sub, roc_thresholds, window=2)
        auc_score = auc(fpr, tpr)
        brier_score = brier_score_loss(flatten_true_batch_time_sub, flatten_batch_pred_sub)

        gmeans = np.sqrt(tpr * (1 - fpr))
        best_gmeans = np.nanmax(gmeans)
        best_gmeans_threshold_index = np.nanargmax(gmeans)
        best_gmeans_threshold = roc_thresholds[best_gmeans_threshold_index]

        J = tpr - fpr
        best_J = np.nanmax(J)
        best_J_threshold_index = np.nanargmax(J)
        best_J_threshold = roc_thresholds[best_J_threshold_index]

        fscore = (2 * precision * recall) / (precision + recall)
        best_fscore = np.nanmax(fscore)
        best_fscore_threshold_index = np.nanargmax(fscore)
        best_fscore_threshold = pr_thresholds[best_fscore_threshold_index]

        stats = [baseline_name, "", i, auc_score, brier_score, best_gmeans, best_gmeans_threshold, best_J, best_J_threshold, best_fscore, best_fscore_threshold]
        model_stats.append(stats)
        auc_formatted = "{:.2f}".format(auc_score)
        fig.add_trace(
            go.Scatter(
                name=f'{baseline_name} - {auc_formatted}',
                mode="lines",
                x=fpr,
                y=tpr,
            )
        )
    fig.write_html(os.path.join(base_path, f'plots/ROC-Curve-6-features-batch-{data_name}-{i}.html'))
model_stats = pd.DataFrame(model_stats, columns=stats_columns)
model_stats.to_csv(os.path.join(base_path, f'plots/stats-{data_name}.csv'))

(156, 99, 3, 1)
(46332,)
dict_keys(['dBarcelona-h4-ch8-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch8-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch8-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch16-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch16-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch16-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch32-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch32-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h4-ch32-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch8-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch8-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch8-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch16-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch16-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch16-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch32-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch32-n3-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h8-ch32-n4-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h16-ch8-n2-sc10-lr0.001-s0.1-do0.2', 'dBarcelona-h16-ch8-n3-sc10-lr0.001-s0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/loc

0.062120189061444966
Barcelona - hidden 16 - context hidden 8 - #layers 3
0.0921409214092141
Barcelona - hidden 16 - context hidden 8 - #layers 4
0.07861271676300578
Barcelona - hidden 16 - context hidden 16 - #layers 2
0.09120521172638435
Barcelona - hidden 16 - context hidden 16 - #layers 3
0.09717868338557994
Barcelona - hidden 16 - context hidden 16 - #layers 4
0.07881773399014778
Barcelona - hidden 16 - context hidden 32 - #layers 2
0.09615384615384616
Barcelona - hidden 16 - context hidden 32 - #layers 3
0.09649122807017543
Barcelona - hidden 16 - context hidden 32 - #layers 4
0.09285714285714286
Barcelona - hidden 32 - context hidden 8 - #layers 2
0.11183355006501951
Barcelona - hidden 32 - context hidden 8 - #layers 3
0.05732484076433122
Barcelona - hidden 32 - context hidden 8 - #layers 4
0.11353711790393012
Barcelona - hidden 32 - context hidden 16 - #layers 2
0.07894736842105263
Barcelona - hidden 32 - context hidden 16 - #layers 3
0.09036860879904876
Barcelona - hidden 32 -

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/loc

batch 1
Barcelona - hidden 4 - context hidden 8 - #layers 2
0.03853820598006645
Barcelona - hidden 4 - context hidden 8 - #layers 3
0.02344747533372026
Barcelona - hidden 4 - context hidden 8 - #layers 4
0.021685701893884635
Barcelona - hidden 4 - context hidden 16 - #layers 2
0.02729885057471264
Barcelona - hidden 4 - context hidden 16 - #layers 3
0.05809859154929579
Barcelona - hidden 4 - context hidden 16 - #layers 4
0.048183556405353725
Barcelona - hidden 4 - context hidden 32 - #layers 2
0.03140007388252678
Barcelona - hidden 4 - context hidden 32 - #layers 3
0.03802281368821293
Barcelona - hidden 4 - context hidden 32 - #layers 4
0.019890081130594088
Barcelona - hidden 8 - context hidden 8 - #layers 2
0.07414272474513438
Barcelona - hidden 8 - context hidden 8 - #layers 3
0.08789237668161436
Barcelona - hidden 8 - context hidden 8 - #layers 4
0.10583941605839417
Barcelona - hidden 8 - context hidden 16 - #layers 2
0.08
Barcelona - hidden 8 - context hidden 16 - #layers 3
0.086230

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/loc

batch 2
Barcelona - hidden 4 - context hidden 8 - #layers 2
0.015101043748612038
Barcelona - hidden 4 - context hidden 8 - #layers 3
0.015579964166082415
Barcelona - hidden 4 - context hidden 8 - #layers 4
0.015543626311994232
Barcelona - hidden 4 - context hidden 16 - #layers 2
0.01834336486099169
Barcelona - hidden 4 - context hidden 16 - #layers 3
0.04317386231038506
Barcelona - hidden 4 - context hidden 16 - #layers 4
0.0288521848212419
Barcelona - hidden 4 - context hidden 32 - #layers 2
0.023873151612328527
Barcelona - hidden 4 - context hidden 32 - #layers 3
0.06358381502890173
Barcelona - hidden 4 - context hidden 32 - #layers 4
0.01378224059854302
Barcelona - hidden 8 - context hidden 8 - #layers 2
0.10897435897435898
Barcelona - hidden 8 - context hidden 8 - #layers 3
0.09090909090909093
Barcelona - hidden 8 - context hidden 8 - #layers 4
0.0821917808219178
Barcelona - hidden 8 - context hidden 16 - #layers 2
0.05072463768115942
Barcelona - hidden 8 - context hidden 16 - #lay

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/loc

0.05194805194805195
Barcelona - hidden 16 - context hidden 16 - #layers 3
0.10810810810810811
Barcelona - hidden 16 - context hidden 16 - #layers 4
0.060606060606060594
Barcelona - hidden 16 - context hidden 32 - #layers 2
0.05249343832020998
Barcelona - hidden 16 - context hidden 32 - #layers 3
0.1016949152542373
Barcelona - hidden 16 - context hidden 32 - #layers 4
0.11981566820276499
Barcelona - hidden 32 - context hidden 8 - #layers 2
0.05695142378559463
Barcelona - hidden 32 - context hidden 8 - #layers 3
0.07185628742514971
Barcelona - hidden 32 - context hidden 8 - #layers 4
0.0688497061293031
Barcelona - hidden 32 - context hidden 16 - #layers 2
0.11274509803921569
Barcelona - hidden 32 - context hidden 16 - #layers 3
0.10313901345291479
Barcelona - hidden 32 - context hidden 16 - #layers 4
0.10655737704918034
Barcelona - hidden 32 - context hidden 32 - #layers 2
0.07427055702917772
Barcelona - hidden 32 - context hidden 32 - #layers 3
0.10714285714285714
Barcelona - hidden 32 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning:

invalid value encountered in true_divide

/usr/loc

In [ ]:
print(model_stats)

                                            model_name  \
0    Barcelona - hidden 4 - context hidden 8 - #lay...   
1    Barcelona - hidden 4 - context hidden 8 - #lay...   
2    Barcelona - hidden 4 - context hidden 8 - #lay...   
3    Barcelona - hidden 4 - context hidden 16 - #la...   
4    Barcelona - hidden 4 - context hidden 16 - #la...   
..                                                 ...   
112  Barcelona - hidden 32 - context hidden 32 - #l...   
113  Barcelona - hidden 32 - context hidden 32 - #l...   
114                             by_batch_mean_baseline   
115                            one_batch_mean_baseline   
116                                     naive_baseline   

                                         model_code  batch  auc_score  \
0      dBarcelona-h4-ch8-n2-sc10-lr0.001-s0.1-do0.2      0   0.630659   
1      dBarcelona-h4-ch8-n3-sc10-lr0.001-s0.1-do0.2      0   0.502292   
2      dBarcelona-h4-ch8-n4-sc10-lr0.001-s0.1-do0.2      0   0.491351   
3     dBarc

In [ ]:
no_baseline_stats = model_stats[model_stats.model_code != '']
top_5_auc_per_batches = []
bad_5_auc_per_batches = []
for i in range(nb_sub_batches):
    batch_stats = no_baseline_stats[no_baseline_stats.batch == i]
    top_5_batch = batch_stats.sort_values('auc_score', ascending=False).iloc[0:5]
    top_5_auc_per_batches.append(top_5_batch)

batch_2_stats = no_baseline_stats[no_baseline_stats.batch == 2]
top_5_auc_batch_2 = batch_2_stats.sort_values('auc_score', ascending=False).iloc[0:10]
print(top_5_auc_batch_2)

                                            model_name  \
100  Barcelona - hidden 16 - context hidden 16 - #l...   
111  Barcelona - hidden 32 - context hidden 32 - #l...   
104  Barcelona - hidden 16 - context hidden 32 - #l...   
98   Barcelona - hidden 16 - context hidden 8 - #la...   
112  Barcelona - hidden 32 - context hidden 32 - #l...   
107  Barcelona - hidden 32 - context hidden 8 - #la...   
109  Barcelona - hidden 32 - context hidden 16 - #l...   
113  Barcelona - hidden 32 - context hidden 32 - #l...   
110  Barcelona - hidden 32 - context hidden 16 - #l...   
89   Barcelona - hidden 8 - context hidden 8 - #lay...   

                                         model_code  batch  auc_score  \
100  dBarcelona-h16-ch16-n3-sc10-lr0.001-s0.1-do0.2      2   0.873797   
111  dBarcelona-h32-ch32-n2-sc10-lr0.001-s0.1-do0.2      2   0.873757   
104  dBarcelona-h16-ch32-n4-sc10-lr0.001-s0.1-do0.2      2   0.867094   
98    dBarcelona-h16-ch8-n4-sc10-lr0.001-s0.1-do0.2      2   0.866808

In [ ]:
print(true_time_sub.shape)
true_time_sub = np.squeeze(true_time_sub)
flatten_true_time_sub = true_time_sub.flatten()
print(flatten_true_time_sub.shape)

match_index = 0

for i in range(nb_sub_batches):

    fig = go.Figure()

    true_batch_time_sub = true_time_sub[match_index, :, i]
    #print(true_batch_time_sub)

    for model in top_5_auc_batch_2.itertuples() :
        model_name = model.model_name
        model_code = model.model_code
        if model_code in model_predictions.keys() :
            batch_pred_sub = model_predictions[model_code]['pred_time_sub']
            batch_pred_sub = batch_pred_sub[match_index, :, i]
            norm = np.linalg.norm(batch_pred_sub)
            batch_pred_sub = batch_pred_sub/norm

            fig.add_trace(
                go.Scatter(
                    name=f"{model_name} - AUC : {model.auc_score:.4f}",
                    mode="lines",
                    y=batch_pred_sub,
                )
            )
    for baseline_name in ['by_batch_mean_baseline', 'one_batch_mean_baseline', 'naive_baseline'] :
        batch_pred_sub = model_predictions[baseline_name]['pred_time_sub']
        batch_pred_sub = batch_pred_sub[match_index, :, i]

        #print(batch_pred_sub.shape)

        #print(batch_pred_sub)

        fig.add_trace(
            go.Scatter(
                name=f"{baseline_name}",
                mode="lines",
                y=batch_pred_sub,
            )
        )
    fig.write_html(os.path.join(base_path, f'plots/average-probabilities-6-features-batch-{data_name}-{i}.html'))
    fig.show()

(156, 99, 3)
(46332,)


In [ ]:


for i in range(nb_sub_batches) :

    z = []
    names = []
    

    true_mean_sub = np.sum(true_time_sub[:, :, i], axis=0)
    norm = np.linalg.norm(true_mean_sub)
    true_mean_sub = true_mean_sub/norm
    print(true_mean_sub.shape)
    batch_pred_sub = model_predictions['by_batch_mean_baseline']['pred_time_sub']
    batch_mean_sub = np.mean(batch_pred_sub[:, :, i], axis=0)
    norm = np.linalg.norm(batch_mean_sub)
    batch_mean_sub = batch_mean_sub/norm
    print(batch_mean_sub.shape)

    z.append(true_mean_sub)
    names.append("Ground Truth")
    z.append(batch_mean_sub)
    names.append("by_batch_mean_baseline")

    top_5_auc_per_batch = top_5_auc_per_batches[i]
    for model in top_5_auc_per_batch.itertuples() :
        model_name = model.model_name
        model_code = model.model_code
        auc_score = model.auc_score
        if model_code in model_predictions.keys() :
            batch_pred_sub = model_predictions[model_code]['pred_time_sub']
            batch_pred_sub = np.sum(batch_pred_sub[:, :, i], axis=0)
            norm = np.linalg.norm(batch_pred_sub)
            batch_pred_sub = batch_pred_sub/norm

            z.append(batch_pred_sub)
            names.append(f'{model_name} - AUC = {auc_score:.4f}'.replace("Barcelona - ", ""))

    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z=z,
        colorscale="blues",
    ))

    for j, name in enumerate(names) :
        fig.add_annotation(x=1, y=j,
            text=name,
            font={
                'size' : 10
            },
            showarrow=False,
            xshift=0,
            xanchor="left"
        )
    
    fig.update_layout(xaxis = {
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': True,
        'dtick' : 15,
        'title' : 'Minute'
    })
    fig.update_yaxes(showticklabels=False)
    fig.update_layout({
        'width': 1000,
        'height' : 400,
        'margin' : {
            'l' : 30,
            'r' : 10,
            't' : 50,
            'b' : 10,
        }
    })

    fig.update_layout(title={
        'text': f"Frequency of substitution batch {i + 1} depending on minutes in La Liga's dataset",
        'y':0.93,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'middle'})
    fig.show()
    fig.write_html(os.path.join(base_path, f'plots/heatmap{i}.html'))
    fig.write_image(os.path.join(base_path, f'plots/heatmap{i}.pdf'))

(99,)
(99,)


(99,)
(99,)


(99,)
(99,)


In [ ]:
warnings.filterwarnings('ignore')
_, _, game_ids = extract_true_data(time_test_dataset)
print(game_ids)
print(time_test_dataset.num_players)
def get_islands(seq):
    difs = np.argwhere(np.diff(seq) != 0)
    islands_indices = np.split(np.arange(seq.size), difs[:, 0] + 1)
    islands = np.split(seq, difs[:, 0] + 1)
    indices_one_islands = list(filter(lambda x: islands[x[0]].any(), enumerate(islands_indices)))
    l = list(zip(*indices_one_islands))
    return list(l[1])

def create_example_game(game_index, model, batch_num):
    game_id = game_ids[game_index]
    game_info = games[games.game_id == game_id].squeeze()
    season_info = seasons[seasons.season_id == game_info.season_id].squeeze()
    teams = SBL.teams(game_id=game_id)
    home_team_name = teams[teams.team_id == game_info.home_team_id].squeeze().team_name
    away_team_name = teams[teams.team_id == game_info.away_team_id].squeeze().team_name
    game_players = SBL.players(game_id)
    model_name = model.model_name.replace("Barcelona - ", "")
    title = f'{season_info.competition_name} - {season_info.season_name} - {home_team_name} v {away_team_name} - {model_name} - batch {batch_num + 1}'
    model_code = model.model_code
    best_model_stats = model_stats[(model_stats.model_code == model_code)]
    best_model_stats = best_model_stats[(best_model_stats.batch == 2)].iloc[0]

    threshold = best_model_stats.fscore_threshold

    true_sub_time = np.squeeze(true_time_sub[game_index][:, batch_num])
    true_sub_player = true_player_sub[game_index][:, batch_num]
    pred_sub_time = np.squeeze(model_predictions[model_code].pred_time_sub[game_index][:, batch_num])
    pred_sub_player = model_predictions[model_code].pred_player_sub[game_index][:, batch_num]
    
    annotations = []

    fig = go.Figure()

    fig.add_trace(go.Scatter(y=pred_sub_time,
                            mode='lines',
                            name='predgdfgdfhgfhfdghfgdhdgf'))
    
    fig.add_hline(y=threshold,
                  line_dash="dot",
                  line_color="green",
                  line_width=0.6)

    annotations.append(dict(
        x=7,
        y=threshold+0.02,
        text=f"Threshold value = {threshold:.3f}",
        font=dict(size=10),
        axref="x",
        ayref="y",
        showarrow=False,
    ))
                  
    subs = np.squeeze(np.argwhere(true_sub_time), axis = -1)
    
    if subs.size != 0 :
        for i, sub in enumerate(subs):
            fig.add_vline(
                x=sub,
                line_color="Black",
                opacity=1,
                name="sub",
                # layer="below",
                line_width=5,
            )

            out_player_name = name_dict[true_sub_player[sub][player_out_index]]
            in_player_name = name_dict[true_sub_player[sub][player_in_index]]
            
            annotations.append(dict(
                x=sub,
                y=0.8 - 0.1 * i,
                text=f'OUT : {out_player_name} <br> IN : {in_player_name}',
                font=dict(size=13),
                align="left",
                axref="x",
                ayref="y",
                arrowcolor='rgb(0, 118, 186)',
                arrowhead=6,
                arrowsize=3,
                arrowwidth=1,
                ax=15,
                ay=0.8 - 0.1 * i
            ))
            if i == 0:
                annotations.append(dict(
                    x=sub,
                    y=1,
                    text=f'True substitution',
                    font=dict(size=10),
                    align="left",
                    ax=0,
                    ay=-15,
                    arrowcolor='rgb(0, 118, 186)',
                    arrowhead=1,
                    arrowsize=1,
                    arrowwidth=1,
                ))

    true_pred = np.where(pred_sub_time >= threshold, 1, 0)
    pred_subs = np.squeeze(np.argwhere(true_pred))
    if pred_subs != []:
        pred_islands = get_islands(true_pred)
    else:
        pred_islands = []
    pred_player_subs = np.array([['Messi', 'Suarez'], ['Cavani', 'Ibrahimovic'], ['Pirlo', 'Marquez']])
    for i, pred_island in enumerate(pred_islands):
        fig.add_vrect(
            x0=pred_island[0] if len(pred_island) > 1 else pred_island[0] - 0.5,
            x1=pred_island[-1] if len(pred_island) > 1 else pred_island[0] + 0.5,
            fillcolor="rgb(159, 206, 138)",
            opacity=0.2,
            layer="below",
            line_width=0,
        )
        
        island_pred_subs = pred_sub_player[pred_island].astype(int)

        most_player_out_index = np.argmax(np.bincount(island_pred_subs[:, player_out_index]))
        most_player_in_index = np.argmax(np.bincount(island_pred_subs[:, player_in_index]))

        out_player_name = name_dict[most_player_out_index]
        in_player_name = name_dict[most_player_in_index]
        

        annotations.append(dict(
            x=np.mean(pred_island),
            y=0.5 - 0.1 * i,
            text=f'OUT : {out_player_name} <br> IN : {in_player_name}',
            font=dict(size=13),
            align="left",
            axref="x",
            ayref="y",
            arrowcolor='rgb(0, 118, 186)',
            arrowhead=7,
            arrowsize=2,
            arrowwidth=0.5,
            ax=15,
            ay=0.5 - 0.1 * i
        ))
        if i == 1:
            annotations.append(dict(
                x=np.mean(pred_island),
                y=1,
                text=f'Predicted substitution',
                font=dict(size=10),
                align="left",
                ax=0,
                ay=-15,
                arrowcolor='rgb(0, 118, 186)',
                arrowhead=1,
                arrowsize=1,
                arrowwidth=1,
            ))

    fig.update_layout(annotations=annotations)
    fig.update_layout(title_text=title,
                    template="plotly_white",
                    title_font=dict(family="Helvetica Neue", size=20),
                    font=dict(family="Helvetica Neue", size=15),
                    showlegend=True)
    fig.update_xaxes(title_text='Time',
                    titlefont=dict(family="Helvetica Neue", size=17),
                    title_standoff=20,
                    tick0=0,
                    range=[0, true_sub_time.shape[0]],
                    dtick=2)
    fig.update_yaxes(title_text='Substitution probability',
                    titlefont=dict(family="Helvetica Neue", size=17),
                    tick0=0,
                    dtick=0.05,
                    range=[0, 1]
                    )
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.99),
        showlegend=False
    )

    fig.update_layout(width=1000,
                    height=600, )
    #fig.show()
    fig.write_html(os.path.join(base_path, f'plots/example-game-{game_index}-{model_code}-{batch_num}.html'))
    fig.write_image(os.path.join(base_path, f'plots/game-{game_index}-{model_code}-{batch_num}.pdf'))

for i in range(nb_sub_batches) :
    for model in top_5_auc_per_batches[i].itertuples() :
        for j in tqdm(range(len(game_ids))):
            create_example_game(j, model, i)

In [ ]:










print(gfdg)
#model_stats = {}
mean_time_sub = np.squeeze(np.mean(baseline_time_sub, axis=0))
baseline_mean_time_sub = np.broadcast_to(mean_time_sub, (true_time_sub.shape[0], true_time_sub.shape[1]))

flatten_mean_baseline_time_sub = baseline_mean_time_sub.flatten()
flatten_time_sub = true_time_sub.flatten()

baseline = np.zeros((true_time_sub.shape[0], true_time_sub.shape[1]))
baseline[:, 45] = 1
baseline[:, 55] = 1
baseline[:, 67] = 1
baseline[:, 75] = 1
baseline[:, 84] = 1
flatten_naive_baseline_time_sub = baseline.flatten()

#model_names = []
model_names = list(model_predictions.keys())
#model_names = model_names[0:3]
model_names.append('naive_baseline')
model_names.append('mean_baseline')

for model_name in model_names:
    print('=====')
    print(f'Model = {model_name}')
    if model_name == 'naive_baseline':
        flatten_time_out = flatten_naive_baseline_time_sub
        pred_time_sub = baseline
    elif model_name == 'mean_baseline':
        flatten_time_out = flatten_mean_baseline_time_sub
        pred_time_sub = baseline_mean_time_sub
    else:
        flatten_time_out = model_predictions[model_name].pred_time_sub.flatten()
        pred_time_sub = model_predictions[model_name].pred_time_sub
    fpr, tpr, roc_thresholds = roc_curve(flatten_time_sub, flatten_time_out, drop_intermediate=True)
    precision, recall, pr_thresholds = precision_recall_curve(flatten_time_sub, flatten_time_out)
    fpr, tpr, precision, recall = dtw_roc_curve(true_time_sub, pred_time_sub, roc_thresholds, window=2)
    auc_score = auc(fpr, tpr)

    brier_score = brier_score_loss(flatten_time_sub, flatten_time_out)

    gmeans = np.sqrt(tpr * (1 - fpr))
    best_gmeans = np.max(gmeans)
    best_gmeans_threshold_index = np.argmax(gmeans)
    best_gmeans_threshold = roc_thresholds[best_gmeans_threshold_index]

    J = tpr - fpr
    best_J = np.max(J)
    best_J_threshold_index = np.argmax(J)
    best_J_threshold = roc_thresholds[best_J_threshold_index]

    fscore = (2 * precision * recall) / (precision + recall)
    best_fscore = np.max(fscore)
    best_fscore_threshold_index = np.argmax(fscore)
    best_fscore_threshold = pr_thresholds[best_fscore_threshold_index]

    f = open(os.path.join(base_path, f'stats-{base_model_name}.txt'), "w")
    f.write('=====')
    f.write(f'Model = {model_name}')
    f.write(f'Gmeans = {gmeans[best_gmeans_threshold_index]}')
    f.write(f'Gmeans Threshold = {best_gmeans_threshold}')
    f.write(f'J = {J[best_J_threshold_index]}')
    f.write(f'J Threshold = {best_J_threshold}')
    f.write(f'FScore = {fscore[best_fscore_threshold_index]}')
    f.write(f'FScore Threshold = {best_fscore_threshold}')
    f.write(f'Brier Score = {brier_score}')
    f.write(f'ROC AUC Score = {auc_score}')

    gmeans = np.sqrt(tpr * (1 - fpr))
    best_gmeans = np.max(gmeans)
    best_gmeans_threshold_index = np.argmax(gmeans)
    best_gmeans_threshold = roc_thresholds[best_gmeans_threshold_index]

    J = tpr - fpr
    best_J = np.max(J)
    best_J_threshold_index = np.argmax(J)
    best_J_threshold = roc_thresholds[best_J_threshold_index]

    fscore = (2 * precision * recall) / (precision + recall)
    best_fscore = np.max(fscore)
    best_fscore_threshold_index = np.argmax(fscore)
    best_fscore_threshold = pr_thresholds[best_fscore_threshold_index]

    print(f'Gmeans = {gmeans[best_gmeans_threshold_index]}')
    print(f'Gmeans Threshold = {best_gmeans_threshold}')
    print(f'J = {J[best_J_threshold_index]}')
    print(f'J Threshold = {best_J_threshold}')
    print(f'FScore = {fscore[best_fscore_threshold_index]}')
    print(f'FScore Threshold = {best_fscore_threshold}')
    print(f'Brier Score = {brier_score}')
    print(f'ROC AUC Score = {auc_score}')

    model_stat = {}
    model_stat = dotdict(model_stat)
    model_stat.auc_score = auc_score
    model_stat.fpr = fpr
    model_stat.tpr = tpr
    model_stat.precision = precision
    model_stat.roc_thresholds = roc_thresholds
    model_stat.pr_thresholds = pr_thresholds
    model_stat.brier_score = brier_score
    model_stat.gmeans = best_gmeans
    model_stat.gmeans_threshold = best_gmeans_threshold
    model_stat.best_gmeans_threshold_index = best_gmeans_threshold_index
    model_stat.J = best_J
    model_stat.J_threshold = best_J_threshold
    model_stat.best_J_threshold_index = best_J_threshold_index
    model_stat.fscore = best_fscore
    model_stat.fscore_threshold = best_fscore_threshold
    model_stat.best_fscore_threshold_index = best_fscore_threshold_index
    model_stats[model_name] = model_stat

In [ ]:
from plotly.subplots import make_subplots

model_files = ['test_new_ROC_mask_mode_time_player_prediction_model-256-2-7-MaskMode.MAX_MINUTE_SIZE-0.01-0.1-3',
               'test_new_ROC_mask_mode_time_player_prediction_model-256-2-7-MaskMode.LAST_MINUTE-0.01-0.1-1']
best_model = 'test_new_ROC_mask_mode_time_player_prediction_model-256-2-7-MaskMode.MAX_MINUTE_SIZE-0.01-0.1-3'
mean_time_sub = np.squeeze(np.mean(true_time_sub, axis=0))

fig = make_subplots(rows=2, cols=1, subplot_titles=("ROC Curve", "Model Profiles"), row_heights=[0.7, 0.3])

fig.add_trace(
    go.Scatter(
        name=f"Naive Baseline - ROC AUC score : {model_stats['naive_baseline'].auc_score:.2f}",
        mode="lines",
        x=model_stats['naive_baseline'].fpr,
        y=model_stats['naive_baseline'].tpr,
        legendgroup="roc",
        legendgrouptitle_text="ROC Curve",
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        name=f"Mean Baseline - ROC AUC score : {model_stats['mean_baseline'].auc_score:.2f}",
        x=model_stats['mean_baseline'].fpr,
        y=model_stats['mean_baseline'].tpr,
        legendgroup="roc",
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        name="Mean probability profile",
        x=np.arange(mean_time_sub.shape[0]),
        y=mean_time_sub,
        legendgroup="profile",
        legendgrouptitle_text="Model Profiles",
    ),
    row=2,
    col=1
)

for i, model_file in enumerate(model_files):
    training_config = training_configs[i]
    print(f'Base name : {training_config.model_name}')
    #model_files = list()
    #for file in os.listdir(modelfolder):
    #    if file.startswith(f'{training_config.model_name}'):
    #        model_files.append(file)
    #training_config.model_files = model_files
    print(training_config)
    #auc_scores = []
    #brier_scores = []
    #for model_file in model_files :
    #    auc_scores.append(model_stats[model_file].auc_score)
    #    brier_scores.append(model_stats[model_file].brier_score)
    #print(auc_scores)
    #print(brier_scores)
    #sorted_auc = [i[0] for i in sorted(enumerate(auc_scores), key=lambda x:x[1])]

    if training_config.mask_mode == MaskMode.MAX_MINUTE_SIZE:
        printed_model_name = 'Model All Minutes'
    else:
        printed_model_name = 'Model Last Minute'

    fig.add_trace(
        go.Scatter(
            name=f"{printed_model_name} - AUC score : {model_stats[model_file].auc_score:.2f}",
            x=model_stats[model_file].fpr,
            y=model_stats[model_file].tpr,
            legendgroup="roc",
        ),
        row=1,
        col=1
    )

    mean_pred_sub = np.squeeze(np.mean(model_predictions[model_file].pred_time_sub, axis=0))
    fig.add_trace(
        go.Scatter(
            name=printed_model_name,
            y=mean_pred_sub,
            legendgroup="profile",
        ),
        row=2,
        col=1
    )

fig.update_xaxes(title_text='False Positive Rate',
                 titlefont=dict(family="Helvetica Neue", size=17),
                 title_standoff=20,
                 tick0=0,
                 dtick=0.05,
                 range=[0, 1],
                 row=1,
                 col=1
                 )

fig.update_yaxes(title_text='True Positive Rate',
                 titlefont=dict(
                     family="Helvetica Neue",
                     size=17),
                 tick0=0,
                 dtick=0.05,
                 range=[0, 1],
                 row=1,
                 col=1
                 )

fig.update_xaxes(title_text='Time',
                 titlefont=dict(family="Helvetica Neue", size=17),
                 title_standoff=20,
                 tick0=0,
                 dtick=2,
                 range=[0, mean_time_sub.shape[0]],
                 row=2,
                 col=1
                 )
fig.update_yaxes(title_text='Substitution probability',
                 titlefont=dict(
                     family="Helvetica Neue",
                     size=17),
                 tick0=0,
                 dtick=0.1,
                 row=2,
                 col=1
                 )

fig.update_layout(height=1000,
                  width=1000,
                  template="plotly_white",
                  title_font=dict(family="Helvetica Neue", size=27),
                  font=dict(family="Helvetica Neue", size=15),
                  showlegend=True)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=0.25,
        xanchor="left",
        x=0.23,
        font=dict(family="Helvetica Neue", size=10),
        groupclick="toggleitem"
    ),
)

fig.write_html(os.path.join(base_path, f'plots/ROC-curve-best.html'))
fig.write_image(os.path.join(base_path, f'plots/ROC-curve-best.pdf'))
fig.show()
